In [28]:
import collections
import pathlib
import re
import string
import pickle
import tensorflow as tf
import pandas as pd
import numpy as np
import random

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

In [29]:
from grabber import ChatBoxGrabber
dataset = []
for key, data in pickle.load(open('data.pickle', 'rb')).items():
    dataset.append(tuple([data.message, data.user]))
random.shuffle(dataset)

In [3]:
for user in {pair[1] for pair in dataset}:
    if sum([True for pair in dataset if pair[1] == user]) < 2000:
        dataset = [pair for pair in dataset if not user == pair[1]]

In [4]:
dataset = [pair for pair in dataset if len(pair[0]) < 100]

In [5]:
messages = [pair[0] for pair in dataset]
senders = [pair[1] for pair in dataset]

In [6]:
vocab = {'':0}
for message in messages:
    for char in message:
        if char not in vocab.keys():
            vocab[char] = len(vocab)

In [7]:
userVocab = {}
for user in senders:
    if user not in userVocab.keys():
            userVocab[user] = len(userVocab)
senders = [userVocab[user] for user in senders]

In [8]:
def vectorize(string):
    if not type(string) == str:
        return [vectorize(str) for str in string]
    ans=np.zeros(100, dtype = int)
    for i in range(len(string)):
        ans[i] = vocab[string[i]]
    return ans
#vectorize(messages[0])

In [9]:
vectors = np.array([vectorize(message) for message in messages])
#dataframe = pd.DataFrame({'x':messages, 'y':senders}).sample(frac=1).reset_index(drop=True)
vectors.shape

(105886, 100)

In [10]:
test = tf.data.Dataset.from_tensor_slices((vectors[:10000], senders[:10000]))
validate = tf.data.Dataset.from_tensor_slices((vectors[10000:30000], senders[10000:30000]))
train = tf.data.Dataset.from_tensor_slices((vectors[30000:], senders[30000:]))

In [11]:
dataset = tf.data.Dataset.from_tensor_slices((messages, np.array(senders)))

In [12]:
model = tf.keras.Sequential([
      layers.Embedding(len(vocab), 64, mask_zero=True),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(len({i for i in senders}))
  ])
model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

In [13]:
train

<TensorSliceDataset shapes: ((100,), ()), types: (tf.int32, tf.int32)>

In [14]:
model.fit(vectors[30000:].tolist(), senders[30000:], epochs=50)

Epoch 1/50
2372/2372 [==============================] - 14s 6ms/step - loss: 2.1907 - accuracy: 0.2420
Epoch 2/50
2372/2372 [==============================] - 15s 6ms/step - loss: 1.8411 - accuracy: 0.3660
Epoch 3/50
2372/2372 [==============================] - 17s 7ms/step - loss: 1.7423 - accuracy: 0.3998
Epoch 4/50
2372/2372 [==============================] - 17s 7ms/step - loss: 1.6765 - accuracy: 0.4209
Epoch 5/50
2372/2372 [==============================] - 17s 7ms/step - loss: 1.6322 - accuracy: 0.4358
Epoch 6/50
2372/2372 [==============================] - 17s 7ms/step - loss: 1.5791 - accuracy: 0.4568
Epoch 7/50
2372/2372 [==============================] - 16s 7ms/step - loss: 1.5352 - accuracy: 0.4698
Epoch 8/50
2372/2372 [==============================] - 15s 7ms/step - loss: 1.4998 - accuracy: 0.4847
Epoch 9/50
2372/2372 [==============================] - 17s 7ms/step - loss: 1.4611 - accuracy: 0.5004
Epoch 10/50
2372/2372 [==============================] - 18s 7ms/step - l

In [15]:
val = []
for user in userVocab.values():
    vects = []
    for i in range(len(senders)):
        if(senders[i] == user):
            vects.append(vectors[i])
    temp = sum(model.predict(np.array(random.sample(vects, 10))))
    print(max(temp) == temp[user])
    if not max(temp) == temp[user]:
        val.append(len(vects))
max(val)

True
True
True
True
True
True
True
True
True
True
True
True
True


ValueError: max() arg is an empty sequence

In [40]:
lil = [messages[i] for i in range(len(senders)) if print(i, end="\r") or senders[i] == userVocab[2903]]

In [57]:
print(len(lil))
lilPres = [predicts[userVocab[2903]] for predicts in model.predict(np.array(vectorize(lil)))]

15230


In [59]:
lilPairs = [(lilPres[i], lil[i]) for i in range(len(lil))]
lilPairs.sort()
lilPairs[-10:]

[(15.597538, 'قیمت اون سه تا دونه عینک بابام: قیمت نهایی: ۱,۰۳۹,۴۰۰ تومان'),
 (15.712978, 'اما همه ی اسرارتو باید بریزی رو داریه.دایره.درایه.حالا هرچی!!!'),
 (15.8894205,
  '@Louis وگرنه من سوپرمن،فلش،جوکر،هارلی،آکوامن...همه ی اینا رو دوست دارم'),
 (15.973396,
  '*با تلفن حرف زدن*نه نه...ناپلئونی دوست ندارم....زبون؟نه....نون خامه ای؟آره....به اندازه باشه.....ببین من آبرو دارما!واسه خودمم یه کیلو کیک برونی بزار کنار میام میبرم....خدافظ'),
 (16.424473,
  '@شرلوک\u200cهلمز پس خیلات راحت.این کرونا نیست......پس به من بدین نمره ی بیست!:21:'),
 (16.807104,
  '*از خواب پریدن*میستری!میستری زنده ست!تو جهنمه!میتونیم برش گردونیم!'),
 (17.057531, '@DarkGuy چچه؟!*اکسیو یخچال**وینگاردیم لویوسا**دیفندو*'),
 (17.378464,
  '@DarkGuy ریموس لوپین،نیمفادورا تانکس،سیریوس بلک،الستور مودی،فرد ویزلی و خیلیای دیگه نباید می مردن'),
 (18.445826, '*نعره زدن*هموند!بسه!اینقدر گاز نده!'),
 (18.524721, 'من،جان ویک،جان سینا،ترمیناتور،ویل مارکس')]

In [64]:
[i for i in range(15) if model.predict(np.array(vectorize(["بچه ها"])))[0][i] > 0]

[0, 1, 3, 5, 6, 7, 10, 12]

In [23]:
userVocab.keys()

dict_keys([2274, 3074, 1034, 2903, 986, 635, 1714, 2803, 2905, 2914, 2454, 2883, 321])

In [90]:
counter = 0
for i in range(0, len(senders)):
        if(senders[i] == 3):
            counter +=1
counter

2777

In [54]:
model.predict(np.array([vectorize("@شرلوک‌هلمز اسمش چیه؟")]))[0]

array([ 3.259055  , -1.9234855 ,  3.8764307 ,  2.1480486 ,  4.610029  ,
        1.1548915 , -0.19314751,  1.8464661 ,  5.452761  ,  1.2430861 ,
        0.9013407 , -0.1955685 , -4.0964465 ], dtype=float32)

In [20]:
sum(model.predict(np.array([vectorize(messages[i]) for i in range(30000) if senders[i] == 2])))

array([  2240.3018,  -5910.2153,   3454.702 ,  -4566.301 , -11807.172 ,
       -10405.755 , -10313.312 ,  -6268.75  ,    845.2459,  -4315.7764,
        -7612.003 , -13785.518 ,  -4810.502 ], dtype=float32)

In [27]:
sum([senders[i]==594 for i in range(30000)])

0

In [32]:
sum([True for pair in dataset if pair[1] == 2507])

1546